In [1]:
#imports
#numpy, pandas, scipy, math, matplotlib 
import numpy as np 
import pandas as pd 
import scipy 
from math import sqrt 
import matplotlib.pyplot as plt

#estimators 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#feature selection
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

#model metrics 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score

#cross validation 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV

In [2]:
rawData = pd.read_csv('trainingData.csv', header=0)
rawData.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


In [3]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19937 entries, 0 to 19936
Columns: 529 entries, WAP001 to TIMESTAMP
dtypes: float64(2), int64(527)
memory usage: 80.5 MB


In [4]:
rawData.describe()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
count,19937.000000,19937.000000,19937.0,19937.0,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,...,19937.0,19937.000000,1.993700e+04,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,1.993700e+04
mean,99.823644,99.820936,100.0,100.0,99.613733,97.130461,94.733661,93.820234,94.693936,99.163766,...,100.0,-7464.275947,4.864871e+06,1.674575,1.212820,148.429954,1.833024,9.068014,13.021869,1.371421e+09
std,5.866842,5.798156,0.0,0.0,8.615657,22.931890,30.541335,33.010404,30.305084,12.634045,...,0.0,123.402010,6.693318e+01,1.223078,0.833139,58.342106,0.372964,4.988720,5.362410,5.572054e+05
min,-97.000000,-90.000000,100.0,100.0,-97.000000,-98.000000,-99.000000,-98.000000,-98.000000,-99.000000,...,100.0,-7691.338400,4.864746e+06,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.369909e+09
25%,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7594.737000,4.864821e+06,1.000000,0.000000,110.000000,2.000000,5.000000,8.000000,1.371056e+09
50%,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7423.060900,4.864852e+06,2.000000,1.000000,129.000000,2.000000,11.000000,13.000000,1.371716e+09
75%,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7359.193000,4.864930e+06,3.000000,2.000000,207.000000,2.000000,13.000000,14.000000,1.371721e+09
max,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7300.818990,4.865017e+06,4.000000,2.000000,254.000000,2.000000,18.000000,24.000000,1.371738e+09


In [6]:
corrData = rawData.corr()
print(corrData)

                    WAP001    WAP002  WAP003  WAP004    WAP005    WAP006  \
WAP001            1.000000 -0.000928     NaN     NaN -0.001348 -0.003762   
WAP002           -0.000928  1.000000     NaN     NaN -0.001385 -0.003865   
WAP003                 NaN       NaN     NaN     NaN       NaN       NaN   
WAP004                 NaN       NaN     NaN     NaN       NaN       NaN   
WAP005           -0.001348 -0.001385     NaN     NaN  1.000000 -0.005610   
...                    ...       ...     ...     ...       ...       ...   
SPACEID           0.016777 -0.035616     NaN     NaN  0.008474 -0.146094   
RELATIVEPOSITION -0.013458 -0.013827     NaN     NaN  0.043336  0.084678   
USERID           -0.011642  0.049948     NaN     NaN -0.023651  0.037283   
PHONEID           0.000123 -0.005633     NaN     NaN  0.020221  0.134668   
TIMESTAMP         0.063228  0.020383     NaN     NaN -0.024054 -0.066426   

                    WAP007    WAP008    WAP009    WAP010  ...  WAP520  \
WAP001        

Removing unneeded features

In [5]:
newData = rawData.drop(['BUILDINGID', 'SPACEID', 'RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP'], axis=1)

In [6]:
newData.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520,LONGITUDE,LATITUDE,FLOOR
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-7541.2643,4.864921e+06,2
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-7536.6212,4.864934e+06,2
2,100,100,100,100,100,100,100,-97,100,100,...,100,100,100,100,100,100,100,-7519.1524,4.864950e+06,2
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-7524.5704,4.864934e+06,2
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-7632.1436,4.864982e+06,0


Train/Test Datasets on sample size - default

In [7]:
#random sampling to 10,000 observations
tenData = newData.sample(n=10000)

In [8]:
#select the features
X = tenData.drop(['LONGITUDE', 'LATITUDE', 'FLOOR'], axis=1)
#select dependent variables
y_lon = tenData['LONGITUDE']
y_lat = tenData['LATITUDE']
y_flo = tenData['FLOOR']

In [9]:
#train and test splitting of data for each dependent variable
X_trainlon, X_testlon, y_trainlon, y_testlon = train_test_split(X, y_lon, test_size=0.3, random_state=42)
X_trainlat, X_testlat, y_trainlat, y_testlat = train_test_split(X, y_lat, test_size=0.3, random_state=42)
X_trainflo, X_testflo, y_trainflo, y_testflo = train_test_split(X, y_flo, test_size=0.3, random_state=42)

In [10]:
X_trainflo.shape, X_testflo.shape

((7000, 520), (3000, 520))

In [11]:
#scale features
sc = StandardScaler()
X_trainlon = sc.fit_transform(X_trainlon)
X_testlon = sc.transform(X_testlon)

sc1 = StandardScaler()
X_trainlat = sc1.fit_transform(X_trainlat)
X_testlat = sc1.transform(X_testlat)

sc2 = StandardScaler()
X_trainflo = sc2.fit_transform(X_trainflo)
X_testflo = sc2.transform(X_testflo)

In [12]:
#remove zero variance features
selector = VarianceThreshold()
X_trainlonZ = selector.fit_transform(X_trainlon)
X_testlonZ = selector.transform(X_testlon)

selector1 = VarianceThreshold()
X_trainlatZ = selector1.fit_transform(X_trainlat)
X_testlatZ = selector1.transform(X_testlat)

selector2 = VarianceThreshold()
X_trainfloZ = selector2.fit_transform(X_trainflo)
X_testfloZ = selector2.transform(X_testflo)

In [13]:
#PCA
pca=PCA(n_components=20)
X_trainlonZP = pca.fit_transform(X_trainlonZ)
X_testlonZP = pca.transform(X_testlonZ)

pca1=PCA(n_components=20)
X_trainlatZP = pca1.fit_transform(X_trainlatZ)
X_testlatZP = pca1.transform(X_testlatZ)

pca2=PCA(n_components=20)
X_trainfloZP = pca2.fit_transform(X_trainfloZ)
X_testfloZP = pca2.transform(X_testfloZ)

In [14]:
X_trainfloZP.shape, X_testfloZP.shape

((7000, 20), (3000, 20))

Train/Test Datasets on full data

In [29]:
#select the features
fullX = newData.drop(['LONGITUDE', 'LATITUDE', 'FLOOR'], axis=1)
#select dependent variables
fully_lon = newData['LONGITUDE']
fully_lat = newData['LATITUDE']
fully_flo = newData['FLOOR']

In [30]:
#train and test splitting of data for each dependent variable
fullX_trainlon, fullX_testlon, fully_trainlon, fully_testlon = train_test_split(fullX, fully_lon, test_size=0.3, random_state=42)
fullX_trainlat, fullX_testlat, fully_trainlat, fully_testlat = train_test_split(fullX, fully_lat, test_size=0.3, random_state=42)
fullX_trainflo, fullX_testflo, fully_trainflo, fully_testflo = train_test_split(fullX, fully_flo, test_size=0.3, random_state=42)

In [31]:
fullX_trainflo.shape, fullX_testflo.shape

((13955, 520), (5982, 520))

In [32]:
#scale features
sc = StandardScaler()
fullX_trainlon = sc.fit_transform(fullX_trainlon)
fullX_testlon = sc.transform(fullX_testlon)

sc1 = StandardScaler()
fullX_trainlat = sc1.fit_transform(fullX_trainlat)
fullX_testlat = sc1.transform(fullX_testlat)

sc2 = StandardScaler()
fullX_trainflo = sc2.fit_transform(fullX_trainflo)
fullX_testflo = sc2.transform(fullX_testflo)

*1. LONGITUDE*

Random Forest - default

In [30]:
lonRF = RandomForestRegressor(n_estimators=100)
lonRF.fit(X_trainlon, y_trainlon)
print(cross_val_score(lonRF, X_trainlon, y_trainlon, cv=3))
print(lonRF.score(X_trainlon, y_trainlon))

[0.98991426 0.99057894 0.99008271]
0.9964604029604118


In [34]:
pred_lonRF = lonRF.predict(X_testlon)
print(r2_score(y_testlon, pred_lonRF))
print(sqrt(mean_squared_error(y_testlon, pred_lonRF)))

0.9937575724950743
9.71826915677412


Random Forest - zero var removed & PCA

In [57]:
lonZPRF = RandomForestRegressor(n_estimators=100)
lonZPRF.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPRF, X_trainlonZP, y_trainlon, cv=3))
print(lonZPRF.score(X_trainlonZP, y_trainlon))

[0.99050291 0.99114661 0.99136162]
0.9963682779958274


In [58]:
pred_lonZPRF = lonZPRF.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPRF))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPRF)))

0.9929175080883149
10.351545918881174


Random Forest - tuning on default

In [88]:
lonRFgrid = {'max_depth': [90,100,110], 'n_estimators': [100,200,300]}

In [89]:
searchlonRF = GridSearchCV(lonRF, param_grid=lonRFgrid, cv=3)
searchlonRF.fit(X_trainlon, y_trainlon)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [90]:
searchlonRF.best_params_

{'max_depth': 90, 'n_estimators': 100}

In [93]:
lonRFbest = RandomForestRegressor(max_depth=90, n_estimators=100)
lonRFbest.fit(X_trainlon, y_trainlon)
print(cross_val_score(lonRFbest, X_trainlon, y_trainlon, cv=3))
print(lonRFbest.score(X_trainlon, y_trainlon))

[0.98966136 0.99039661 0.99026782]
0.9964408778975882


In [94]:
pred_lonRFbest = lonRFbest.predict(X_testlon)
print(r2_score(y_testlon, pred_lonRFbest))
print(sqrt(mean_squared_error(y_testlon, pred_lonRFbest)))

0.9937307183891263
9.739150084979926


Random Forest - tuned on zero var & PCA

In [100]:
searchlonZPRF = GridSearchCV(lonZPRF, param_grid=lonRFgrid, cv=3)
searchlonZPRF.fit(X_trainlonZP, y_trainlon)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [101]:
searchlonZPRF.best_params_

{'max_depth': 90, 'n_estimators': 200}

In [102]:
lonZPRFbest = RandomForestRegressor(max_depth=90, n_estimators=200)
lonZPRFbest.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPRFbest, X_trainlonZP, y_trainlon, cv=3))
print(lonZPRFbest.score(X_trainlonZP, y_trainlon))

[0.99045978 0.99124118 0.99130493]
0.9963836676989895


In [103]:
pred_lonZPRFbest = lonZPRFbest.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPRFbest))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPRFbest)))

0.9929287553702317
10.343323317570295


Gradient Boosting - default 

In [35]:
lonGB = GradientBoostingRegressor(n_estimators=100, random_state=42)
lonGB.fit(X_trainlon, y_trainlon)
print(cross_val_score(lonGB, X_trainlon, y_trainlon, cv=3))
print(lonGB.score(X_trainlon, y_trainlon))

[0.96689507 0.96663857 0.96231681]
0.9698315852235502


In [36]:
pred_lonGB = lonGB.predict(X_testlon)
print(r2_score(y_testlon, pred_lonGB))
print(sqrt(mean_squared_error(y_testlon, pred_lonGB)))

0.9652546014513239
22.927717298353567


Gradient Boosting - zero var removed & PCA

In [59]:
lonZPGB = GradientBoostingRegressor(n_estimators=100, random_state=42)
lonZPGB.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPGB, X_trainlonZP, y_trainlon, cv=3))
print(lonZPGB.score(X_trainlonZP, y_trainlon))

[0.9869227  0.98720282 0.98740928]
0.989297818139762


In [60]:
pred_lonZPGB = lonZPGB.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPGB))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPGB)))

0.9887749781095302
13.031840853366157


Gradient Boosting - tuning on zero var & PCA (since better performance metrics)

In [72]:
lonGBgrid = {'learning_rate': [0.05, 0.1, 0.5, 1],'max_depth': [90,100,110], 'min_samples_leaf': [3,4],  'n_estimators': [100,200,300]}

In [73]:
searchlonGB = GridSearchCV(lonZPGB, param_grid=lonGBgrid, cv=3)
searchlonGB.fit(X_trainlonZP, y_trainlon)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_n...
                            

In [78]:
searchlonGB.best_params_

{'learning_rate': 0.05,
 'max_depth': 90,
 'min_samples_leaf': 4,
 'n_estimators': 300}

In [83]:
lonZPGBbest = GradientBoostingRegressor(learning_rate=0.05, max_depth=90, min_samples_leaf=4, n_estimators=300, random_state=42)
lonZPGBbest.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPGBbest, X_trainlonZP, y_trainlon, cv=3))
print(lonZPGBbest.score(X_trainlonZP, y_trainlon))

[0.99004014 0.99065055 0.99027037]
0.9970627635653898


In [84]:
pred_lonZPGBbest = lonZPGBbest.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPGBbest))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPGBbest)))

0.9922037983967984
10.860597027636782


SVR - default

In [38]:
lonSVR = SVR()
lonSVR.fit(X_trainlon, y_trainlon)
print(cross_val_score(lonSVR, X_trainlon, y_trainlon, cv=3))
print(lonSVR.score(X_trainlon, y_trainlon))

[0.74811921 0.7129368  0.74974392]
0.8080386546192088


In [39]:
pred_lonSVR = lonSVR.predict(X_testlon)
print(r2_score(y_testlon, pred_lonSVR))
print(sqrt(mean_squared_error(y_testlon, pred_lonSVR)))

0.8009773323726201
54.87359845047181


SVR - zero var removed & PCA

In [61]:
lonZPSVR = SVR()
lonZPSVR.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPSVR, X_trainlonZP, y_trainlon, cv=3))
print(lonZPSVR.score(X_trainlonZP, y_trainlon))

[0.92809402 0.91823464 0.92560066]
0.9409014404497621


In [62]:
pred_lonZPSVR = lonZPSVR.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPSVR))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPSVR)))

0.937213631256749
30.82086708042501


KNN - default

In [43]:
lonKNN = KNeighborsRegressor(n_neighbors=2)
lonKNN.fit(X_trainlon, y_trainlon)
print(cross_val_score(lonKNN, X_trainlon, y_trainlon, cv=3))
print(lonKNN.score(X_trainlon, y_trainlon))

[0.9898174  0.9900331  0.99084207]
0.9953737298396631


In [44]:
pred_lonKNN = lonKNN.predict(X_testlon)
print(r2_score(y_testlon, pred_lonKNN))
print(sqrt(mean_squared_error(y_testlon, pred_lonKNN)))

0.9931636358277817
10.170089247802489


KNN - zero var removed & PCA

In [63]:
lonZPKNN = KNeighborsRegressor(n_neighbors=2)
lonZPKNN.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPKNN, X_trainlonZP, y_trainlon, cv=3))
print(lonZPKNN.score(X_trainlonZP, y_trainlon))

[0.9889034  0.99003052 0.98632953]
0.9951177551497624


In [66]:
pred_lonZPKNN = lonZPKNN.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPKNN))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPKNN)))

0.9930920574036719
10.22319228445524


KNN - tuning on default

In [75]:
lonKNNgrid = {'n_neighbors':[2,4,6,8,10], 'weights':['uniform','distance']}

In [76]:
searchlonKNN = GridSearchCV(lonKNN, param_grid=lonKNNgrid, cv=3)
searchlonKNN.fit(X_trainlon, y_trainlon)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=2, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 4, 6, 8, 10],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [77]:
searchlonKNN.best_params_

{'n_neighbors': 2, 'weights': 'distance'}

In [86]:
lonKNNbest = KNeighborsRegressor(weights='distance', n_neighbors=2)
lonKNNbest.fit(X_trainlon, y_trainlon)
print(cross_val_score(lonKNNbest, X_trainlon, y_trainlon, cv=3))
print(lonKNNbest.score(X_trainlon, y_trainlon))

[0.99033531 0.99025052 0.99092057]
0.9967047398043601


In [87]:
pred_lonKNNbest = lonKNNbest.predict(X_testlon)
print(r2_score(y_testlon, pred_lonKNNbest))
print(sqrt(mean_squared_error(y_testlon, pred_lonKNNbest)))

0.9932075735642424
10.137354669103663


KNN - tuned on zero var & PCA

In [96]:
searchlonZPKNN = GridSearchCV(lonZPKNN, param_grid=lonKNNgrid, cv=3)
searchlonZPKNN.fit(X_trainlonZP, y_trainlon)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=2, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 4, 6, 8, 10],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [97]:
searchlonZPKNN.best_params_

{'n_neighbors': 6, 'weights': 'distance'}

In [98]:
lonZPKNNbest = KNeighborsRegressor(n_neighbors=6, weights='distance')
lonZPKNNbest.fit(X_trainlonZP, y_trainlon)
print(cross_val_score(lonZPKNNbest, X_trainlonZP, y_trainlon, cv=3))
print(lonZPKNNbest.score(X_trainlonZP, y_trainlon))

[0.9913841  0.99135062 0.99041194]
0.9970505962605071


In [99]:
pred_lonZPKNNbest = lonZPKNNbest.predict(X_testlonZP)
print(r2_score(y_testlon, pred_lonZPKNNbest))
print(sqrt(mean_squared_error(y_testlon, pred_lonZPKNNbest)))

0.9933111550436385
10.059762648778337


RF (best model) on full Longitude data

In [33]:
lonRFfull = RandomForestRegressor(max_depth=90, n_estimators=100)
lonRFfull.fit(fullX_trainlon, fully_trainlon)
print(cross_val_score(lonRFfull, fullX_trainlon, fully_trainlon, cv=3))
print(lonRFfull.score(fullX_trainlon, fully_trainlon))

[0.99414438 0.99441686 0.99442061]
0.9973557407275191


In [34]:
pred_lonRFfull = lonRFfull.predict(fullX_testlon)
print(r2_score(fully_testlon, pred_lonRFfull))
print(sqrt(mean_squared_error(fully_testlon, pred_lonRFfull)))

0.9948520813743131
8.864778477468587


*2. LATITUDE*

Random Forest - default

In [95]:
latRF = RandomForestRegressor(n_estimators=100)
latRF.fit(X_trainlat, y_trainlat)
print(cross_val_score(latRF, X_trainlat, y_trainlat, cv=3))
print(latRF.score(X_trainlat, y_trainlat))

[0.97944033 0.98168852 0.98379544]
0.9942708569867656


In [104]:
pred_latRF = latRF.predict(X_testlat)
print(r2_score(y_testlat, pred_latRF))
print(sqrt(mean_squared_error(y_testlat, pred_latRF)))

0.9862684181613927
7.7978831617998345


Random Forest - zero var removed & PCA

In [111]:
latZPRF = RandomForestRegressor(n_estimators=100)
latZPRF.fit(X_trainlatZP, y_trainlat)
print(cross_val_score(latZPRF, X_trainlatZP, y_trainlat, cv=3))
print(latZPRF.score(X_trainlatZP, y_trainlat))

[0.97801208 0.98183008 0.97983277]
0.9937546221984859


In [112]:
pred_latZPRF = latZPRF.predict(X_testlatZP)
print(r2_score(y_testlat, pred_latZPRF))
print(sqrt(mean_squared_error(y_testlat, pred_latZPRF)))

0.9822952333751351
8.854449027450132


Random Forest - tuned on default

In [130]:
latRFgrid = {'max_depth': [90,100,110], 'n_estimators': [100,200,300]}

In [131]:
searchlatRF = GridSearchCV(latRF, param_grid=latRFgrid, cv=3)
searchlatRF.fit(X_trainlat, y_trainlat)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [132]:
searchlatRF.best_params_

{'max_depth': 100, 'n_estimators': 300}

In [133]:
latRFbest = RandomForestRegressor(max_depth=100, n_estimators=300)
latRFbest.fit(X_trainlat, y_trainlat)
print(cross_val_score(latRFbest, X_trainlat, y_trainlat, cv=3))
print(latRFbest.score(X_trainlat, y_trainlat))

[0.97951421 0.98164318 0.98421523]
0.9943215987483627


In [134]:
pred_latRFbest = latRFbest.predict(X_testlat)
print(r2_score(y_testlat, pred_latRFbest))
print(sqrt(mean_squared_error(y_testlat, pred_latRFbest)))

0.9867856752693428
7.64960352432136


Gradient Boosting - default

In [105]:
latGB = GradientBoostingRegressor(n_estimators=100, random_state=42)
latGB.fit(X_trainlat, y_trainlat)
print(cross_val_score(latGB, X_trainlat, y_trainlat, cv=3))
print(latGB.score(X_trainlat, y_trainlat))

[0.94741818 0.94933978 0.94769927]
0.9537393150490763


In [106]:
pred_latGB = latGB.predict(X_testlat)
print(r2_score(y_testlat, pred_latGB))
print(sqrt(mean_squared_error(y_testlat, pred_latGB)))

0.9467294812208028
15.358892547726914


Gradient Boosting - zero var removed & PCA

In [19]:
latZPGB = GradientBoostingRegressor(n_estimators=100, random_state=42)
latZPGB.fit(X_trainlatZP, y_trainlat)
print(cross_val_score(latZPGB, X_trainlatZP, y_trainlat, cv=3))
print(latZPGB.score(X_trainlatZP, y_trainlat))

[0.97426167 0.97259795 0.96941817]
0.9762110820417252


In [20]:
pred_latZPGB = latZPGB.predict(X_testlatZP)
print(r2_score(y_testlat, pred_latZPGB))
print(sqrt(mean_squared_error(y_testlat, pred_latZPGB)))

0.9738823974950241
10.951537296199614


Gradient Boosting - tuned on zero var removed & PCA

In [21]:
latGBgrid = {'learning_rate': [0.05, 0.1, 0.5, 1],'max_depth': [90,100,110], 'min_samples_leaf': [3,4],  'n_estimators': [100,200,300]}

In [22]:
searchlatGB = GridSearchCV(latZPGB, param_grid=latGBgrid, cv=3)
searchlatGB.fit(X_trainlatZP, y_trainlat)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_n...
                            

In [23]:
searchlatGB.best_params_

{'learning_rate': 0.05,
 'max_depth': 90,
 'min_samples_leaf': 4,
 'n_estimators': 200}

In [24]:
latZPGBbest = GradientBoostingRegressor(learning_rate=0.05, max_depth=90, min_samples_leaf=4, n_estimators=200, random_state=42)
latZPGBbest.fit(X_trainlatZP, y_trainlat)
print(cross_val_score(latZPGBbest, X_trainlatZP, y_trainlat, cv=3))
print(latZPGBbest.score(X_trainlatZP, y_trainlat))

[0.98233063 0.98171266 0.97917821]
0.996910116318198


In [25]:
pred_latZPGBbest = latZPGBbest.predict(X_testlatZP)
print(r2_score(y_testlat, pred_latZPGBbest))
print(sqrt(mean_squared_error(y_testlat, pred_latZPGBbest)))

0.9839765623886593
8.578004691905043


SVR - default

In [107]:
latSVR = SVR()
latSVR.fit(X_trainlat, y_trainlat)
print(cross_val_score(latSVR, X_trainlat, y_trainlat, cv=3))
print(latSVR.score(X_trainlat, y_trainlat))

[0.73646016 0.71713362 0.73702153]
0.7830674186503042


In [108]:
pred_latSVR = latSVR.predict(X_testlat)
print(r2_score(y_testlat, pred_latSVR))
print(sqrt(mean_squared_error(y_testlat, pred_latSVR)))

0.7782851815916303
31.333852817137743


SVR - zero var removed & PCA

In [115]:
latZPSVR = SVR()
latZPSVR.fit(X_trainlatZP, y_trainlat)
print(cross_val_score(latZPSVR, X_trainlatZP, y_trainlat, cv=3))
print(latZPSVR.score(X_trainlatZP, y_trainlat))

[0.90391929 0.89882154 0.90186022]
0.9218432228662168


In [116]:
pred_latZPSVR = latZPSVR.predict(X_testlatZP)
print(r2_score(y_testlat, pred_latZPSVR))
print(sqrt(mean_squared_error(y_testlat, pred_latZPSVR)))

0.9204339837144749
18.770690378699058


KNN - default

In [109]:
latKNN = KNeighborsRegressor(n_neighbors=2)
latKNN.fit(X_trainlat, y_trainlat)
print(cross_val_score(latKNN, X_trainlat, y_trainlat, cv=3))
print(latKNN.score(X_trainlat, y_trainlat))

[0.97576719 0.98290294 0.98097598]
0.9924696350513256


In [110]:
pred_latKNN = latKNN.predict(X_testlat)
print(r2_score(y_testlat, pred_latKNN))
print(sqrt(mean_squared_error(y_testlat, pred_latKNN)))

0.9864531832482124
7.74524334045701


KNN - zero var removed & PCA

In [119]:
latZPKNN = KNeighborsRegressor(n_neighbors=2)
latZPKNN.fit(X_trainlatZP, y_trainlat)
print(cross_val_score(latZPKNN, X_trainlatZP, y_trainlat, cv=3))
print(latZPKNN.score(X_trainlatZP, y_trainlat))

[0.97849382 0.98271606 0.97820438]
0.9907566134929093


In [120]:
pred_latZPKNN = latZPKNN.predict(X_testlatZP)
print(r2_score(y_testlat, pred_latZPKNN))
print(sqrt(mean_squared_error(y_testlat, pred_latZPKNN)))

0.9845455218696484
8.272630199889274


KNN - tuned on default

In [121]:
latKNNgrid = {'n_neighbors':[2,4,6,8,10], 'weights':['uniform','distance']}

In [122]:
searchlatKNN = GridSearchCV(latKNN, param_grid=latKNNgrid, cv=3)
searchlatKNN.fit(X_trainlat, y_trainlat)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=2, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 4, 6, 8, 10],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [123]:
searchlatKNN.best_params_

{'n_neighbors': 2, 'weights': 'distance'}

In [124]:
latKNNbest = KNeighborsRegressor(weights='distance', n_neighbors=2)
latKNNbest.fit(X_trainlat, y_trainlat)
print(cross_val_score(latKNNbest, X_trainlat, y_trainlat, cv=3))
print(latKNNbest.score(X_trainlat, y_trainlat))

[0.97667358 0.98349458 0.98125084]
0.9952159290549014


In [125]:
pred_latKNNbest = latKNNbest.predict(X_testlat)
print(r2_score(y_testlat, pred_latKNNbest))
print(sqrt(mean_squared_error(y_testlat, pred_latKNNbest)))

0.9868497820659606
7.63102566298852


KNN - tuned on zero var & PCA

In [126]:
searchlatZPKNN = GridSearchCV(latZPKNN, param_grid=latKNNgrid, cv=3)
searchlatZPKNN.fit(X_trainlatZP, y_trainlat)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=2, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 4, 6, 8, 10],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [127]:
searchlatZPKNN.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [128]:
latZPKNNbest = KNeighborsRegressor(n_neighbors=4, weights='distance')
latZPKNNbest.fit(X_trainlatZP, y_trainlat)
print(cross_val_score(latZPKNNbest, X_trainlatZP, y_trainlat, cv=3))
print(latZPKNNbest.score(X_trainlatZP, y_trainlat))

[0.97989955 0.98326808 0.98152471]
0.9952910440512699


In [129]:
pred_latZPKNNbest = latZPKNNbest.predict(X_testlatZP)
print(r2_score(y_testlat, pred_latZPKNNbest))
print(sqrt(mean_squared_error(y_testlat, pred_latZPKNNbest)))

0.9858991718460415
7.902031359836009


KNN (best model) on full Latitude data

In [36]:
latKNNfull = KNeighborsRegressor(weights='distance', n_neighbors=2)
latKNNfull.fit(fullX_trainlat, fully_trainlat)
print(cross_val_score(latKNNfull, fullX_trainlat, fully_trainlat, cv=3))
print(latKNNfull.score(fullX_trainlat, fully_trainlat))

[0.98677936 0.98587918 0.98587479]
0.9952126840636782


In [37]:
pred_latKNNfull = latKNNfull.predict(fullX_testlat)
print(r2_score(fully_testlat, pred_latKNNfull))
print(sqrt(mean_squared_error(fully_testlat, pred_latKNNfull)))

0.9851355364709194
8.129667086974786


*3. FLOOR (ALTITUDE)*

Random Forest - default

In [27]:
floRF = RandomForestClassifier(n_estimators=100)
floRF.fit(X_trainflo, y_trainflo)
print(cross_val_score(floRF, X_trainflo, y_trainflo, cv=3))

[0.98800343 0.98799829 0.97813973]


In [31]:
pred_floRF = floRF.predict(X_testflo)
print(classification_report(y_testflo, pred_floRF))
print(confusion_matrix(y_testflo, pred_floRF))
print(cohen_kappa_score(y_testflo, pred_floRF))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       619
           1       0.99      0.99      0.99       807
           2       0.99      0.98      0.98       678
           3       0.98      0.99      0.99       725
           4       1.00      0.99      1.00       171

    accuracy                           0.99      3000
   macro avg       0.99      0.99      0.99      3000
weighted avg       0.99      0.99      0.99      3000

[[615   0   0   4   0]
 [  4 799   4   0   0]
 [  0   7 664   7   0]
 [  0   0   5 720   0]
 [  0   0   0   1 170]]
0.986187712157101


Random Forest - zero var removed & PCA

In [47]:
floZPRF = RandomForestClassifier(n_estimators=100)
floZPRF.fit(X_trainfloZP, y_trainflo)
print(cross_val_score(floZPRF, X_trainfloZP, y_trainflo, cv=3))

[0.90959726 0.92456065 0.90441492]


In [48]:
pred_floZPRF = floZPRF.predict(X_testfloZP)
print(classification_report(y_testflo, pred_floZPRF))
print(confusion_matrix(y_testflo, pred_floZPRF))
print(cohen_kappa_score(y_testflo, pred_floZPRF))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       619
           1       0.93      0.94      0.94       807
           2       0.92      0.88      0.90       678
           3       0.88      0.93      0.91       725
           4       0.95      0.83      0.89       171

    accuracy                           0.92      3000
   macro avg       0.93      0.91      0.91      3000
weighted avg       0.92      0.92      0.92      3000

[[585  34   0   0   0]
 [ 30 757  17   3   0]
 [  2  19 597  60   0]
 [  5   1  36 676   7]
 [  0   0   0  29 142]]
0.8949193127348279


Random Forest - tuned on default

In [61]:
floRFgrid = {'max_depth': [90,100,110], 'n_estimators': [100,200,300]}

In [63]:
searchfloRF = GridSearchCV(floRF, param_grid=floRFgrid, cv=3)
searchfloRF.fit(X_trainflo, y_trainflo)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [64]:
searchfloRF.best_params_

{'max_depth': 90, 'n_estimators': 200}

In [65]:
floRFbest = RandomForestClassifier(max_depth=90, n_estimators=200)
floRFbest.fit(X_trainflo, y_trainflo)
print(cross_val_score(floRFbest, X_trainflo, y_trainflo, cv=3))

[0.98757498 0.98842692 0.98114016]


In [66]:
pred_floRFbest = floRFbest.predict(X_testflo)
print(classification_report(y_testflo, pred_floRFbest))
print(confusion_matrix(y_testflo, pred_floRFbest))
print(cohen_kappa_score(y_testflo, pred_floRFbest))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       619
           1       1.00      0.99      0.99       807
           2       0.99      0.98      0.99       678
           3       0.99      0.99      0.99       725
           4       1.00      0.99      1.00       171

    accuracy                           0.99      3000
   macro avg       0.99      0.99      0.99      3000
weighted avg       0.99      0.99      0.99      3000

[[619   0   0   0   0]
 [  4 799   4   0   0]
 [  0   4 667   7   0]
 [  4   0   2 719   0]
 [  0   0   0   1 170]]
0.9887792822970017


Gradient Boosting - default

In [15]:
floGB = GradientBoostingClassifier(n_estimators=100, random_state=42)
floGB.fit(X_trainflo, y_trainflo)
print(cross_val_score(floGB, X_trainflo, y_trainflo, cv=3))

[0.95886889 0.95970853 0.95885126]


In [16]:
pred_floGB = floGB.predict(X_testflo)
print(classification_report(y_testflo, pred_floGB))
print(confusion_matrix(y_testflo, pred_floGB))
print(cohen_kappa_score(y_testflo, pred_floGB))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       641
           1       0.95      0.96      0.96       765
           2       0.96      0.93      0.94       701
           3       0.95      0.97      0.96       736
           4       0.99      0.96      0.97       157

    accuracy                           0.96      3000
   macro avg       0.97      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000

[[628  10   3   0   0]
 [ 13 736  15   1   0]
 [  0  15 651  34   1]
 [  1  11   8 715   1]
 [  0   0   0   6 151]]
0.9485804458204976


Gradient Boosting - zero var removed & PCA

In [49]:
floZPGB = GradientBoostingClassifier(n_estimators=100, random_state=42)
floZPGB.fit(X_trainfloZP, y_trainflo)
print(cross_val_score(floZPGB, X_trainfloZP, y_trainflo, cv=3))

[0.8697515  0.89069867 0.86712387]


In [50]:
pred_floZPGB = floZPGB.predict(X_testfloZP)
print(classification_report(y_testflo, pred_floZPGB))
print(confusion_matrix(y_testflo, pred_floZPGB))
print(cohen_kappa_score(y_testflo, pred_floZPGB))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       619
           1       0.89      0.89      0.89       807
           2       0.88      0.85      0.87       678
           3       0.85      0.89      0.87       725
           4       0.93      0.81      0.87       171

    accuracy                           0.89      3000
   macro avg       0.89      0.88      0.88      3000
weighted avg       0.89      0.89      0.89      3000

[[572  43   0   4   0]
 [ 51 722  27   7   0]
 [  0  31 579  68   0]
 [  3  13  51 648  10]
 [  0   0   1  31 139]]
0.8529939511334599


Gradient Boosting - tuned on default

In [17]:
floGBgrid = {'learning_rate': [0.05, 0.1, 0.5],'max_depth': [90,100,110], 'n_estimators': [100,200]}

In [18]:
searchfloGB = GridSearchCV(floGB, param_grid=floGBgrid, cv=3)
searchfloGB.fit(X_trainflo, y_trainflo)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [19]:
searchfloGB.best_params_

{'learning_rate': 0.5, 'max_depth': 110, 'n_estimators': 200}

In [20]:
floGBbest = GradientBoostingClassifier(learning_rate=0.5, max_depth=110, n_estimators=200, random_state=42)
floGBbest.fit(X_trainflo, y_trainflo)
print(cross_val_score(floGBbest, X_trainflo, y_trainflo, cv=3))

[0.9798629  0.9777111  0.95199314]


In [21]:
pred_floGBbest = floGBbest.predict(X_testflo)
print(classification_report(y_testflo, pred_floGBbest))
print(confusion_matrix(y_testflo, pred_floGBbest))
print(cohen_kappa_score(y_testflo, pred_floGBbest))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       641
           1       0.99      0.98      0.99       765
           2       0.99      0.98      0.99       701
           3       0.99      0.99      0.99       736
           4       0.99      0.97      0.98       157

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.99      0.99      3000

[[637   4   0   0   0]
 [  9 752   3   1   0]
 [  0   5 690   5   1]
 [  9   0   1 725   1]
 [  0   0   0   5 152]]
0.9809951952398147


SVC - default

In [24]:
floSVC = SVC()
floSVC.fit(X_trainflo, y_trainflo)
print(cross_val_score(floSVC, X_trainflo, y_trainflo, cv=3))

[0.9335904  0.93184741 0.93656237]


In [35]:
pred_floSVC = floSVC.predict(X_testflo)
print(classification_report(y_testflo, pred_floSVC))
print(confusion_matrix(y_testflo, pred_floSVC))
print(cohen_kappa_score(y_testflo, pred_floSVC))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       619
           1       0.96      0.94      0.95       807
           2       0.92      0.93      0.92       678
           3       0.94      0.95      0.94       725
           4       0.99      0.93      0.96       171

    accuracy                           0.95      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.95      0.95      0.95      3000

[[604  14   1   0   0]
 [ 20 762  19   6   0]
 [  2  16 629  31   0]
 [  5   0  31 688   1]
 [  0   0   3   9 159]]
0.9317758164844309


SVC - zero var removed & PCA

In [51]:
floZPSVC = SVC()
floZPSVC.fit(X_trainfloZP, y_trainflo)
print(cross_val_score(floZPSVC, X_trainfloZP, y_trainflo, cv=3))

[0.87275064 0.88084012 0.86926704]


In [52]:
pred_floZPSVC = floZPSVC.predict(X_testfloZP)
print(classification_report(y_testflo, pred_floZPSVC))
print(confusion_matrix(y_testflo, pred_floZPSVC))
print(cohen_kappa_score(y_testflo, pred_floZPSVC))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       619
           1       0.92      0.89      0.91       807
           2       0.87      0.84      0.85       678
           3       0.82      0.92      0.87       725
           4       0.98      0.73      0.83       171

    accuracy                           0.89      3000
   macro avg       0.90      0.86      0.88      3000
weighted avg       0.89      0.89      0.89      3000

[[582  33   0   4   0]
 [ 50 718  32   7   0]
 [  2  22 567  87   0]
 [  1   4  52 665   3]
 [  0   0   0  47 124]]
0.8510726019622339


SVC - tuned on default

In [22]:
floSVCgrid = {'C': [0.1,1,10], 'kernel':['linear','rbf']}

In [25]:
searchfloSVC = GridSearchCV(floSVC, param_grid=floSVCgrid, cv=3)
searchfloSVC.fit(X_trainflo, y_trainflo)

GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
searchfloSVC.best_params_

{'C': 10, 'kernel': 'rbf'}

In [27]:
floSVCbest = SVC(C=10, kernel='rbf', random_state=42)
floSVCbest.fit(X_trainflo, y_trainflo)
print(cross_val_score(floSVCbest, X_trainflo, y_trainflo, cv=3))

[0.94344473 0.93784826 0.94427775]


In [28]:
pred_floSVCbest = floSVCbest.predict(X_testflo)
print(classification_report(y_testflo, pred_floSVCbest))
print(confusion_matrix(y_testflo, pred_floSVCbest))
print(cohen_kappa_score(y_testflo, pred_floSVCbest))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       641
           1       0.96      0.95      0.95       765
           2       0.92      0.95      0.94       701
           3       0.97      0.94      0.95       736
           4       0.98      0.98      0.98       157

    accuracy                           0.95      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.95      0.95      0.95      3000

[[624  16   1   0   0]
 [ 14 724  25   2   0]
 [  0  12 669  20   0]
 [  9   1  30 693   3]
 [  0   0   0   3 154]]
0.9412772711663053


KNN - default

In [45]:
floKNN = KNeighborsClassifier(n_neighbors=3)
floKNN.fit(X_trainflo, y_trainflo)
print(cross_val_score(floKNN, X_trainflo, y_trainflo, cv=3))

[0.90659811 0.91170167 0.90527218]


In [46]:
pred_floKNN = floKNN.predict(X_testflo)
print(classification_report(y_testflo, pred_floKNN))
print(confusion_matrix(y_testflo, pred_floKNN))
print(cohen_kappa_score(y_testflo, pred_floKNN))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94       619
           1       0.94      0.92      0.93       807
           2       0.93      0.86      0.89       678
           3       0.91      0.93      0.92       725
           4       0.98      0.96      0.97       171

    accuracy                           0.93      3000
   macro avg       0.93      0.93      0.93      3000
weighted avg       0.93      0.93      0.92      3000

[[611   7   1   0   0]
 [ 44 742  15   5   1]
 [ 13  32 580  53   0]
 [  8  11  27 677   2]
 [  0   0   1   5 165]]
0.9029351267278986


KNN - zero var removed & PCA

In [59]:
floZPKNN = KNeighborsClassifier(n_neighbors=3)
floZPKNN.fit(X_trainfloZP, y_trainflo)
print(cross_val_score(floZPKNN, X_trainfloZP, y_trainflo, cv=3))

[0.89717224 0.90784398 0.88812688]


In [60]:
pred_floZPKNN = floZPKNN.predict(X_testfloZP)
print(classification_report(y_testflo, pred_floZPKNN))
print(confusion_matrix(y_testflo, pred_floZPKNN))
print(cohen_kappa_score(y_testflo, pred_floZPKNN))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       619
           1       0.93      0.91      0.92       807
           2       0.90      0.86      0.88       678
           3       0.88      0.91      0.90       725
           4       0.96      0.88      0.92       171

    accuracy                           0.91      3000
   macro avg       0.92      0.91      0.91      3000
weighted avg       0.91      0.91      0.91      3000

[[595  24   0   0   0]
 [ 45 735  23   4   0]
 [  2  26 585  65   0]
 [  8   7  41 662   7]
 [  0   0   2  18 151]]
0.8825406627384004


RF (best model) on full Floor data

In [38]:
floRFfull = RandomForestClassifier(max_depth=90, n_estimators=200)
floRFfull.fit(fullX_trainflo, fully_trainflo)
print(cross_val_score(floRFfull, fullX_trainflo, fully_trainflo, cv=3))

[0.99441101 0.99290628 0.99118469]


In [39]:
pred_floRFfull = floRFfull.predict(fullX_testflo)
print(classification_report(fully_testflo, pred_floRFfull))
print(confusion_matrix(fully_testflo, pred_floRFfull))
print(cohen_kappa_score(fully_testflo, pred_floRFfull))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1316
           1       0.99      1.00      1.00      1492
           2       1.00      0.99      1.00      1357
           3       1.00      0.99      0.99      1479
           4       1.00      0.99      1.00       338

    accuracy                           0.99      5982
   macro avg       1.00      0.99      0.99      5982
weighted avg       0.99      0.99      0.99      5982

[[1311    5    0    0    0]
 [   2 1489    1    0    0]
 [   0    4 1349    4    0]
 [  14    0    1 1464    0]
 [   0    0    0    2  336]]
0.9928687881365095
